In [1]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine, inspect, text

### Database connections

In [2]:
# Cargar configuración desde el archivo config.yml
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones a las bases de datos de fuente y bodega
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear SQLAlchemy connections
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# Extract

### SQL Query

In [3]:
# Definir consulta SQL para extraer datos de novedades del servicio de mensajería
query = """
SELECT 
    id as novedad_id,
    tipo_novedad_id,
    descripcion 
FROM 
    mensajeria_novedadesservicio 
"""

In [4]:
# Leer datos de la fuente en un DataFrame de pandas
dim_novedad = pd.read_sql(query, fuente_conn)

# Transformation

In [5]:
# Agregar novedad de carga
dim_novedad['saved'] = date.today()

### Verifications

In [6]:
# Verificar los datos
print("\nInformación del DataFrame:")
print(dim_novedad.info())
print("\nMuestra de las novedades:")
print(dim_novedad.head())



Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   novedad_id       5208 non-null   int64 
 1   tipo_novedad_id  5208 non-null   int64 
 2   descripcion      5208 non-null   object
 3   saved            5208 non-null   object
dtypes: int64(2), object(2)
memory usage: 162.9+ KB
None

Muestra de las novedades:
   novedad_id  tipo_novedad_id descripcion       saved
0           4                1           A  2024-11-10
1           5                1        Halo  2024-11-10
2           6                1           A  2024-11-10
3           7                1           B  2024-11-10
4           8                1           A  2024-11-10


# Load

In [7]:
# Cargar en la bodega
dim_novedad.to_sql('dim_novedad', bodega_conn, if_exists='replace', index_label='key_dim_novedad')

208